<a href="https://colab.research.google.com/github/EllaThomasson/labs-ds3001/blob/main/Lab_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

In [ ]:
# I did practice writing queries using saferproducts.gov.

# Here is an example:
url = 'https://www.saferproducts.gov/RestWebServices/Recall?format=json&ProductType=Phone'

Question 2

Question 3

Question 4

Question 5